<p style="align: center;"><img align=center src="https://drive.google.com/uc?export=view&id=1I8kDikouqpH4hf7JBiSYAeNT2IO52T-T" width=600 height=480/></p>
<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Весна 2021</b></h3>

# Autoencoders


# Часть 1. Vanilla Autoencoder (10 баллов)

## 1.1. Подготовка данных (0.5 балла)


In [1]:
import numpy as np
from torch.autograd import Variable
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
def fetch_dataset(attrs_name = "lfw_attributes.txt",
                      images_name = "lfw-deepfunneled",
                      dx=80,dy=80,
                      dimx=64,dimy=64
    ):

    #download if not exists
    if not os.path.exists(images_name):
        print("images not found, donwloading...")
        os.system("wget http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz -O tmp.tgz")
        print("extracting...")
        os.system("tar xvzf tmp.tgz && rm tmp.tgz")
        print("done")
        assert os.path.exists(images_name)

    if not os.path.exists(attrs_name):
        print("attributes not found, downloading...")
        os.system("wget http://www.cs.columbia.edu/CAVE/databases/pubfig/download/%s" % attrs_name)
        print("done")

    #read attrs
    df_attrs = pd.read_csv("lfw_attributes.txt",sep='\t',skiprows=1,) 
    df_attrs = pd.DataFrame(df_attrs.iloc[:,:-1].values, columns = df_attrs.columns[1:])


    #read photos
    photo_ids = []
    for dirpath, dirnames, filenames in os.walk(images_name):
        for fname in filenames:
            if fname.endswith(".jpg"):
                fpath = os.path.join(dirpath,fname)
                photo_id = fname[:-4].replace('_',' ').split()
                person_id = ' '.join(photo_id[:-1])
                photo_number = int(photo_id[-1])
                photo_ids.append({'person':person_id,'imagenum':photo_number,'photo_path':fpath})

    photo_ids = pd.DataFrame(photo_ids)
    # print(photo_ids)
    #mass-merge
    #(photos now have same order as attributes)
    df = pd.merge(df_attrs,photo_ids,on=('person','imagenum'))

    assert len(df)==len(df_attrs),"lost some data when merging dataframes"

    # print(df.shape)
    #image preprocessing
    all_photos =df['photo_path'].apply(skimage.io.imread)\
                                .apply(lambda img:img[dy:-dy,dx:-dx])\
                                .apply(lambda img: resize(img,[dimx,dimy]))

    all_photos = np.stack(all_photos.values)#.astype('uint8')
    all_attrs = df.drop(["photo_path","person","imagenum"],axis=1)
    
    return all_photos, all_attrs

In [3]:
# The following line fetches you two datasets: images, usable for autoencoder training and attributes.
# Those attributes will be required for the final part of the assignment (applying smiles), so please keep them in mind

#from datasets import fetch_dataset
import os
import pandas as pd
import skimage.io
from skimage.transform import resize
data, attrs = fetch_dataset()


Разбейте выборку картинок на train и val, выведите несколько картинок в output, чтобы посмотреть, как они выглядят, и приведите картинки к тензорам pytorch, чтобы можно было скормить их сети:

In [4]:
train_photos, val_photos, train_attrs, val_attrs = train_test_split(data, attrs,
                                                                    train_size=0.9, shuffle=True)
train_loader = torch.utils.data.DataLoader(train_photos, batch_size=32)
val_loader = torch.utils.data.DataLoader(val_photos, batch_size=32)
n = 5
genphoto = np.random.choice(len(train_photos), n)
genphot = [i for i in genphoto]
lst = []
for i, j in zip(range(n), genphoto):
    plt.figure()
    #plt.imshow(X_train[j]) 
    plt.imshow(train_photos[j])

## 1.2. Архитектура модели (1.5 балла)
В этом разделе мы напишем и обучем обычный автоэнкодер.



<img src="https://www.notion.so/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F4b8adf79-8e6a-4b7d-9061-8617a00edbb1%2F__2021-04-30__14.53.33.png?table=block&id=56f187b4-279f-4208-b1ed-4bda5f91bfc0&width=2880&userId=3b1b5e32-1cfb-4b0f-8705-5a524a8f56e3&cache=v2" alt="Autoencoder">


^ напомню, что автоэнкодер выглядит вот так

In [6]:
dim_code = 75 # выберите размер латентного вектора

Реализуем autoencoder. Архитектуру (conv, fully-connected, ReLu, etc) можете выбирать сами. Экспериментируйте!

In [7]:
from copy import deepcopy

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.flatten = nn.Flatten()
        self.encoder = nn.Sequential(
        nn.Linear(12288, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, dim_code))
        
        self.decoder = nn.Sequential(
        nn.Linear(dim_code, 128),
        nn.ReLU(),
        nn.Linear(128, 256),
        nn.ReLU(),
        nn.Linear(256, 512),
        nn.ReLU(),
        nn.Linear(512, 12288),
        nn.ReLU())
        
        
    def forward(self, x, z):
        
        #<реализуйте forward проход автоэнкодера
        #в качестве ваозвращаемых переменных -- латентное представление картинки (latent_code) 
        #и полученная реконструкция изображения (reconstruction)>
        
        #return reconstruction, latent_code
        x = self.flatten(x).float()
        return self.decoder(self.encoder(x)), self.encoder(x), self.decoder(z)


In [8]:
criterion = nn.MSELoss()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
autoencoder = Autoencoder().to(device)

optimizer = torch.optim.Adam(autoencoder.parameters())
#print(autoencoder)

## 1.3 Обучение (2 балла)

Осталось написать код обучения автоэнкодера. При этом было бы неплохо в процессе иногда смотреть, как автоэнкодер реконструирует изображения на данном этапе обучения. Наример, после каждой эпохи (прогона train выборки через автоэекодер) можно смотреть, какие реконструкции получились для каких-то изображений val выборки.

А, ну еще было бы неплохо выводить графики train и val лоссов в процессе тренировки =)

In [9]:
train_photos, val_photos, train_attrs, val_attrs = train_test_split(data, attrs,
                                                                    train_size=0.9, shuffle=True)
train_loader = torch.utils.data.DataLoader(train_photos, batch_size=32)
val_loader = torch.utils.data.DataLoader(val_photos, batch_size=32)


In [11]:
n_epochs = 25
train_losses = []
val_losses = []
latent_lst = []
z = torch.tensor(np.random.randn(25, dim_code))

def my_plot(epochs, loss):
    plt.plot(epochs, loss)
    
    
for epoch in tqdm(range(n_epochs)):
    autoencoder.train()
    train_losses_per_epoch = []
    for batch in train_loader:
        #print(batch.to(device).shape)
        optimizer.zero_grad()
        reconstruction, latent_code, z_dist = autoencoder(batch.to(device), z.to(device).float())
        latent_lst.extend(latent_code)
        #print(reconstruction.shape)
        reconstruction = reconstruction.view(-1, 64, 64, 3)
        #print(reconstruction.shape)
        loss = criterion(batch.to(device).float(), reconstruction)
        loss.backward()
        optimizer.step()
        train_losses_per_epoch.append(loss.item())
        #writer.add_scalar('Loss/Train', loss, epoch)
    train_losses.append(np.mean(train_losses_per_epoch))

    autoencoder.eval()
    val_losses_per_epoch = []
    with torch.no_grad():
        for batch in val_loader:
            reconstruction, latent_code, z_val = autoencoder(batch.to(device),z.to(device).float() )
            reconstruction = reconstruction.view(-1, 64, 64, 3)
            loss = criterion(batch.to(device).float(), reconstruction)
            val_losses_per_epoch.append(loss.item())
            #writer.add_scalar('Loss/Validation', loss, epoch)
    val_losses.append(np.mean(val_losses_per_epoch))

In [12]:
epochs__ = range(1,26)
plt.plot(epochs__, train_losses, 'g', label='Training loss')
plt.plot(epochs__, val_losses, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Давайте посмотрим, как наш тренированный автоэекодер кодирует и восстанавливает картинки:

In [15]:
import matplotlib.pyplot as plt



autoencoder.eval()
with torch.no_grad():
    for batch in val_loader:
        reconstruction, laten_code, zdis = autoencoder(batch.to(device), z.to(device).float())
        reconstruction = reconstruction.view(  -1, 64, 64, 3)
        result = reconstruction.cpu().detach().numpy()
        ground_truth = batch.numpy()
        break
        
        
        
plt.figure(figsize=(8, 20))
for i, (gt, res) in enumerate(zip(ground_truth[:10], result[:10])):
    #print(ground_truth[:10])
    plt.subplot(10, 2, 2*i+1)
    plt.imshow(gt)
    plt.subplot(10, 2, 2*i+2)
    plt.imshow(res)

Not bad, right? 

## 1.4. Sampling (2 балла)

Давайте теперь будем не просто брать картинку, прогонять ее через автоэекодер и получать реконструкцию, а попробуем создать что-то НОВОЕ

Давайте возьмем и подсунем декодеру какие-нибудь сгенерированные нами векторы (например, из нормального распределения) и посмотрим на результат реконструкции декодера:

__Подсказка:__Е сли вместо лиц у вас выводится непонятно что, попробуйте посмотреть, как выглядят латентные векторы картинок из датасета. Так как в обучении нейронных сетей есть определенная доля рандома, векторы латентного слоя могут быть распределены НЕ как `np.random.randn(25, <latent_space_dim>)`. А чтобы у нас получались лица при запихивании вектора декодеру, вектор должен быть распределен так же, как латентные векторы реальных фоток. Так что в таком случае придется рандом немного подогнать.

In [16]:
encod  = []
z = torch.tensor(np.random.randn(25, dim_code))
with torch.no_grad():
    for idx, batch in enumerate(train_loader):
        batch = batch.view( batch.size(0), -1)
        #print(batch.shape)
        reconstruction = autoencoder.encoder(batch.to(device).float())
        #reconstruction = reconstruction.view(-1, 64, 64, 3)
        res_z = reconstruction.cpu().detach().numpy()
        ground_truth = batch.numpy()
        encod.extend(res_z)
        #reck.append(res_z)
encod = np.array(encod).T

In [17]:
z_mean = np.mean([encod[i].mean() for i in range(len(encod))])
z_std = np.mean([encod[i].std() for i in range(len(encod))])
print(z_mean, z_std)

In [18]:
z = torch.normal(z_mean, 3*z_std, size = (25, dim_code)) #-0.18094799, 0.505032733
print(z.shape)

In [28]:
autoencoder.eval()
reck = []
gro = []
with torch.no_grad():
    for idx, batch in enumerate(z):
        #print(batch.size())
        reconstruction = autoencoder.decoder(batch.to(device).float())
        reconstruction = reconstruction.view(-1, 64, 64, 3)
        res_z = reconstruction.cpu().detach().numpy()
        ground_truth = batch.numpy()
        reck.append(res_z)
        gro.append(ground_truth)
lstim = []
plt.figure(figsize=(8, 20))
for i, res in enumerate(reck[:15]):
    plt.subplot(15, 2, 2*i+1)
    plt.imshow(res[0])


## Time to make fun! (4 балла)

Давайте научимся пририсовывать людям улыбки =)

<img src="https://i.imgur.com/tOE9rDK.png" alt="linear" width="700" height="400">

План такой:

1. Нужно выделить "вектор улыбки": для этого нужно из выборки изображений найти несколько (~15) людей с улыбками и столько же без.

Найти людей с улыбками вам поможет файл с описанием датасета, скачанный вместе с датасетом. В нем указаны имена картинок и присутствубщие атрибуты (улыбки, очки...)

2. Вычислить латентный вектор для всех улыбающихся людей (прогнать их через encoder) и то же для всех грустненьких

3. Вычислить, собственно, вектор улыбки -- посчитать разность между средним латентным вектором улыбающихся людей и средним латентным вектором грустных людей

4. А теперь приделаем улыбку грустному человеку: добавим полученный в пункте 3 вектор к латентному вектору грустного человека и прогоним полученный вектор через decoder. Получим того же человека, но уже не грустненького!

In [71]:
smile = sorted(train_attrs['Smiling'])[len(train_attrs)-45:] # вместо 15 взял 45 изображений
smile_not = sorted(train_attrs['Smiling'])[2555:2600] 


In [72]:
smile_photo = []
smilenot_photo = []
for i, j in enumerate(train_attrs['Smiling']):
    if j in smile:
        smile_photo.append(i)
    if j in smile_not:
        smilenot_photo.append(i)
        

In [50]:
print(smilenot_photo)

In [ ]:
plt.imshow(train_photos[144])

In [83]:
#print(smile_photo)
encod_smile = []
encod_smilenot = []
#encod_smile = np.array(enc)
train_photos = torch.tensor(train_photos)
with torch.no_grad():
    for idx, nidx in zip(smile_photo, smilenot_photo):
        reconstruction_smile = autoencoder.encoder(torch.tensor(train_photos[idx].view(1, 64*64*3)).to(device).float())
        reconstruction_smilenot = autoencoder.encoder(torch.tensor(train_photos[nidx].view(1, 64*64*3)).to(device).float())   
        encod_smile.extend(reconstruction_smile.cpu().detach().numpy())
        encod_smilenot.extend(reconstruction_smilenot.cpu().detach().numpy())
        

In [84]:
fortrain_smile = encod_smile
fortrain_smilenot = encod_smilenot

In [87]:
encod_smile, encod_smilenot = np.array(encod_smile).T, np.array(encod_smilenot).T

In [91]:
encod_smile_new = [encod_smile[i].mean() for i in range(len(encod_smile))]
encod_smilenot_new = [encod_smilenot[i].mean() for i in range(len(encod_smilenot))]
new_smile = [encod_smile_new[i] - encod_smilenot_new[i] for i in range(dim_code)]
fortrain_smilenot_ = []
for i in range(len(fortrain_smilenot)):
    fortrain_smilenot_.append(fortrain_smilenot[i] + 2 * new_smile[i])

In [92]:
for_smile = []
for_smile_not = []
with torch.no_grad():
    for vector, idx in zip(fortrain_smilenot_, smilenot_photo):
        vec_recon = autoencoder.decoder(torch.tensor(vector).to(device).float())
        vec_recon = vec_recon.view(-1, 64, 64, 3).cpu().detach().numpy()
        for_smile.extend(vec_recon)
        sm_not, enc, dec = autoencoder(torch.tensor(train_photos[idx].view(1, 64*64*3)).to(device).float(), z.to(device).float())
        sm_not = sm_not.view(-1, 64, 64, 3).cpu().detach().numpy()
        for_smile_not.extend(sm_not)
        

plt.figure(figsize=(8, 20))
for i, (res1, res2) in enumerate(zip(for_smile[:15], for_smile_not[:15])):
    plt.subplot(15, 2, 2*i+1)
    plt.imshow(res2)
    plt.subplot(15, 2, 2*i+2)
    plt.imshow(res1)



Вуаля! Вы восхитительны!

Теперь вы можете пририсовывать людям не только улыбки, но и много чего другого -- закрывать/открывать глаза, пририсовывать очки... в общем, все, на что хватит фантазии и на что есть атрибуты в `all_attrs`:)

# Часть 2: Variational Autoencoder (10 баллов) 

Займемся обучением вариационных автоэнкодеров — проапгрейженной версии AE. Обучать будем на датасете MNIST, содержащем написанные от руки цифры от 0 до 9

In [93]:
from torchvision.transforms import *
batch_size = 32
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## 2.1 Архитектура модели и обучение (2 балла)

Реализуем VAE. Архитектуру (conv, fully-connected, ReLu, etc) можете выбирать сами. Рекомендуем пользоваться более сложными моделями, чем та, что была на семинаре:) Экспериментируйте!

In [94]:
class Reshape(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)


class Trim(nn.Module):
    def __init__(self, *args):
        super().__init__()

    def forward(self, x):
        return x[:, :, :28, :28]

In [95]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        '''<определите архитектуры encoder и decoder
        помните, у encoder должны быть два "хвоста", 
        т.е. encoder должен кодировать картинку в 2 переменные -- mu и logsigma>'''

        self.encoder = nn.Sequential(
                nn.Conv2d(1, 32, stride=(1, 1), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Conv2d(32, 64, stride=(2, 2), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Conv2d(64, 64, stride=(2, 2), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Conv2d(64, 64, stride=(1, 1), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Flatten()
                #nn.BatchNorm2d(3136)
        )
        self.z_mean = torch.nn.Linear(3136, 2)
        self.z_log_var = torch.nn.Linear(3136, 2)
        #mu = self.z_mean(self.encoder(x))
        #logsigma = self.z_log_var(self.encoder(x))
        
        '''<реализуйте forward проход энкодера
        в качестве ваозвращаемых переменных -- mu и logsigma>'''
        
        self.decoder = nn.Sequential(
                torch.nn.Linear(2, 3136),
                Reshape(-1, 64, 7, 7),
                nn.ConvTranspose2d(64, 64, stride=(1, 1), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.ConvTranspose2d(64, 64, stride=(2, 2), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.ConvTranspose2d(64, 32, stride=(2, 2), kernel_size=(3, 3), padding=0), 
                nn.LeakyReLU(0.01),
                nn.ConvTranspose2d(32, 1, stride=(1, 1), kernel_size=(3, 3), padding=0), 
                Trim(),  # 1x29x29 -> 1x28x28
                nn.Flatten(),
                nn.Linear(784, 512),#########################3333
                nn.LeakyReLU(0.01),
                nn.Linear(512, 256),
                nn.LeakyReLU(0.01),
                nn.Linear(256, 784),#################################
                nn.Sigmoid()
                )
        #reconstruction = self.decoder(z)
    
    def gaussian_sampler(self, mu, logsigma):
        if self.training:
            eps = torch.randn(mu.size(0), mu.size(1)).to(device)
            z = mu + eps * torch.exp(logsigma/2.) 
            return z
            #<засемплируйте латентный вектор из нормального распределения с параметрами mu и sigma>
        else:
            # на инференсе возвращаем не случайный вектор из нормального распределения, а центральный -- mu. 
            # на инференсе выход автоэнкодера должен быть детерминирован.
            return mu
    

       #<реализуйте forward проход декодера
        #в качестве возвращаемой переменной -- reconstruction>

    def forward(self, x):
        mu, logsigma = self.z_mean(self.encoder(x)), self.z_log_var(self.encoder(x))
        reconstruction = self.decoder(self.gaussian_sampler(mu, logsigma))#decode(gaussian_sampler(mu, logsigma))
        #<используя encode и decode, реализуйте forward проход автоэнкодера
        #в качестве ваозвращаемых переменных -- mu, logsigma и reconstruction>
        return mu, logsigma, reconstruction

Определим лосс и его компоненты для VAE:

Надеюсь, вы уже прочитали материал в towardsdatascience (или еще где-то) про VAE и знаете, что лосс у VAE состоит из двух частей: KL и log-likelihood.

Общий лосс будет выглядеть так:

$$\mathcal{L} = -D_{KL}(q_{\phi}(z|x)||p(z)) + \log p_{\theta}(x|z)$$

Формула для KL-дивергенции:

$$D_{KL} = -\frac{1}{2}\sum_{i=1}^{dimZ}(1+log(\sigma_i^2)-\mu_i^2-\sigma_i^2)$$

В качестве log-likelihood возьмем привычную нам кросс-энтропию.

In [108]:
def KL_divergence(mu, logsigma):

    loss = - 0.5 * torch.sum(1 + logsigma - mu ** 2 - logsigma.exp()) 
    return loss

def log_likelihood(x, reconstruction):

    loss = nn.BCELoss(reduction = 'sum')#<binary cross-entropy>
    return loss(reconstruction, x)

def loss_vae(x, mu, logsigma, reconstruction):
    return KL_divergence(mu, logsigma) + log_likelihood(x, reconstruction)


И обучим модель:

In [109]:
criterion = loss_vae
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

autoencoder = VAE().to(device)
#print(autoencoder)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr = 0.00007) #0.00005

In [ ]:
autoencoder.z_mean(train_loader[0])

In [110]:
n_epochs = 25
train_losses = []
val_losses = []

for epoch in tqdm(range(n_epochs)):
    autoencoder.train()
    train_losses_per_epoch = []
    for batch in train_loader:
        optimizer.zero_grad()
        mu, logsigma, reconstruction = autoencoder(torch.tensor(batch[0]).to(device))
        #print(reconstruction.shape, mu.shape, logsigma.shape, end = '\n' )
        reconstruction = reconstruction.view(32,1,28,28)
        #print(reconstruction.shape)
        loss = criterion(batch[0].to(device).float(), mu, logsigma, reconstruction)
        loss.backward()
        optimizer.step()
        train_losses_per_epoch.append(loss.item())

    train_losses.append(np.mean(train_losses_per_epoch))

    autoencoder.eval()
    val_losses_per_epoch = []
    with torch.no_grad():
        for batch in val_loader:
            mu, logsigma, reconstruction = autoencoder(torch.tensor(batch[0]).to(device))
            #print(reconstruction.shape)
            reconstruction = reconstruction.view(-1, 1, 28, 28)####################
            loss = criterion(batch[0].to(device).float(), mu, logsigma, reconstruction)
            val_losses_per_epoch.append(loss.item())

    val_losses.append(np.mean(val_losses_per_epoch))

In [ ]:
for ep in tqdm(range(25)):
    autoencoder.eval()
    with torch.no_grad():
        for batch_val in val_loader:
            mu, logsigma, reconstruction = autoencoder(torch.tensor(batch_val[0]).to(device))
        #print(batch_val[0].shape)
            reconstruction = reconstruction.view(-1, 1, 28, 28)####################
            loss = criterion(batch_val[0].to(device).float(), mu, logsigma, reconstruction)
            val_losses_per_epoch.append(loss.item())

    val_losses.append(np.mean(val_losses_per_epoch))

In [111]:
epochs__ = range(1,26)
plt.plot(epochs__, train_losses, 'g', label='Training loss')
plt.plot(epochs__, val_losses, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Давайте посмотрим, как наш тренированный VAE кодирует и восстанавливает картинки:

In [112]:
###################3understand
autoencoder.eval()
with torch.no_grad():
    for batch in val_loader:
        encoder_1 = autoencoder.encoder(batch[0].to(device))
        muu = autoencoder.z_mean(encoder_1)
        z_log_varr= autoencoder.z_log_var(encoder_1)
        gaussian = autoencoder.gaussian_sampler(muu, z_log_varr)
        break

In [115]:
result_val = []
ground_truth_val = []
autoencoder.eval()
with torch.no_grad():
    for batch in val_loader:
        mu, logsigma, reconstruction = autoencoder(batch[0].to(device))
        reconstruction = reconstruction.view(-1, 1, 28, 28)
        result = reconstruction.cpu().detach().numpy()
        ground_truth = batch[0].numpy()
        result_val.extend(result)
        ground_truth_val.extend(ground_truth)
        break

In [116]:
import matplotlib.pyplot as plt


plt.figure(figsize=(8, 20))
for i, (gt, res) in enumerate(zip(ground_truth_val[:10], result_val[:10])):
    plt.subplot(10, 2, 2*i+1)
    plt.imshow(gt[0])
    plt.subplot(10, 2, 2*i+2)
    plt.imshow(res[0])

Давайте попробуем проделать для VAE то же, что и с обычным автоэнкодером -- подсунуть decoder'у из VAE случайные векторы из нормального распределения и посмотреть, какие картинки получаются:

In [117]:
# вспомните про замечание из этого же пункта обычного AE про распределение латентных переменных
#z = np.array([np.random.normal(0, 1, 100) for i in range(10)])
encod_vae = []
with torch.no_grad():
    for idx, batch in enumerate(val_loader):
        batch = batch[0]
        mu, logsigma = autoencoder.z_mean(autoencoder.encoder(batch.to(device).float())), autoencoder.z_log_var(autoencoder.encoder(batch.to(device).float()))
        reconstruction = autoencoder.gaussian_sampler(mu, logsigma)
        res_z = reconstruction.cpu().detach().numpy()
        ground_truth = batch.numpy()
        encod_vae.extend(res_z)
        #reck.append(res_z)
encod_vae = np.array(encod_vae)

In [121]:
encod_vae = []
with torch.no_grad():
    for idx, batch in enumerate(val_loader):
        batch = batch[0]
        mu, logsigma = autoencoder.z_mean(autoencoder.encoder(batch.to(device).float())), autoencoder.z_log_var(autoencoder.encoder(batch.to(device).float()))
        reconstruction = autoencoder.gaussian_sampler(mu, logsigma)
        res_z = reconstruction.cpu().detach().numpy()
        ground_truth = batch.numpy()
        encod_vae.extend(res_z)
encod_vae = np.array(encod_vae)


z_mean_vae = torch.tensor(np.mean([encod_vae[0][i] for i in range(len(encod_vae[0]))]))
z_std_vae= torch.tensor(np.mean([encod_vae[1][i] for i in range(len(encod_vae[1]))]))
z_vae = torch.tensor(np.array([np.random.normal(z_mean_vae, abs(2*z_std_vae), 2) for i in range(10)]))
autoencoder.eval()
reck = []
gro = []
with torch.no_grad():
    for i in z_vae:
        reconstruction = autoencoder.decoder(i.to(device).float())
        reconstruction = reconstruction.view(1,1, 28, 28)
        res_z = reconstruction.cpu().detach().numpy()
        reck.append(res_z)
lstim = []
plt.figure(figsize=(8, 20))
for i, res in enumerate(reck[:10]):
    plt.subplot(15, 2, 2*i+1)
    plt.imshow(res[0][0])

## 2.2. Latent Representation (2 балла)

Давайте посмотрим, как латентные векторы картинок лиц выглядят в пространстве.
Ваша задача -- изобразить латентные векторы картинок точками в двумерном просторанстве. 

Это позволит оценить, насколько плотно распределены латентные векторы изображений цифр в пространстве. 

Плюс давайте сделаем такую вещь: покрасим точки, которые соответствуют картинкам каждой цифры, в свой отдельный цвет

Подсказка: красить -- это просто =) У plt.scatter есть параметр c (color), см. в документации.


Итак, план:
1. Получить латентные представления картинок тестового датасета
2. С помощтю `TSNE` (есть в `sklearn`) сжать эти представления до размерности 2 (чтобы можно было их визуализировать точками в пространстве)
3. Визуализировать полученные двумерные представления с помощью `matplotlib.scatter`, покрасить разными цветами точки, соответствующие картинкам разных цифр.

In [122]:
tsne_latent = []
with torch.no_grad():
    for batch in val_loader:
        batch = batch[0]
        mu, logsigma = autoencoder.z_mean(autoencoder.encoder(batch.to(device))), autoencoder.z_log_var(autoencoder.encoder(batch.to(device)))
        latent_vector = autoencoder.gaussian_sampler(mu, logsigma)
        tsne_latent.extend(latent_vector.cpu().detach().numpy())

In [ ]:
!pip install tsne-torch
from tsne_torch import TorchTSNE as TSNE

In [124]:
from sklearn.manifold import TSNE
X_embedded_vae = TSNE(n_components=2).fit_transform(tsne_latent)
#X_embedded = TSNE(n_components=2).fit_transform(encod_vae.T)
#<ваш код получения латентных представлений, применения TSNE и визуализации>

In [ ]:
ddd = [test_dataset[i][1] for i in range(len(test_dataset))]

In [127]:
import pandas as pd
import seaborn as sns
d_vae = [test_dataset[i][1] for i in range(len(test_dataset))]
df = pd.DataFrame()
df["y"] = d_vae
df["xx"] = X_embedded_vae[:,0]
df["yy"] = X_embedded_vae[:,1]

sns.scatterplot(x="xx", y="yy", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 10),
                data=df).set(title="MNIST data T-SNE projection")
 

Что вы думаете о виде латентного представления?

__Congrats v2.0!__

## 2.3. Conditional VAE (6 баллов)


Мы уже научились обучать обычный AE на датасете картинок и получать новые картинки, используя генерацию шума и декодер. 
Давайте теперь допустим, что мы обучили AE на датасете MNIST и теперь хотим генерировать новые картинки с числами с помощью декодера (как выше мы генерили рандомные лица). 
И вот нам понадобилось сгенерировать цифру 8, и мы подставляем разные варианты шума, но восьмерка никак не генерится:(

Хотелось бы добавить к нашему AE функцию "выдай мне рандомное число из вот этого вот класса", где классов десять (цифры от 0 до 9 образуют десять классов).  Conditional AE — так называется вид автоэнкодера, который предоставляет такую возможность. Ну, название "conditional" уже говорит само за себя.

И в этой части задания мы научимся такие обучать.

### Архитектура

На картинке ниже представлена архитектура простого Conditional VAE.

По сути, единственное отличие от обычного -- это то, что мы вместе с картинкой в первом слое энкодера и декодера передаем еще информацию о классе картинки. 

То есть, в первый (входной) слой энкодера подается конкатенация картинки и информации о классе (например, вектора из девяти нулей и одной единицы). В первый слой декодера подается конкатенация латентного вектора и информации о классе.


![alt text](https://sun9-63.userapi.com/impg/Mh1akf7mfpNoprrSWsPOouazSmTPMazYYF49Tw/djoHNw_9KVA.jpg?size=1175x642&quality=96&sign=e88baec5f9bb91c8443fba31dcf0a4df&type=album)

![alt text](https://sun9-73.userapi.com/impg/UDuloLNKhzTBYAKewgxke5-YPsAKyGOqA-qCRg/MnyCavJidxM.jpg?size=1229x651&quality=96&sign=f2d21bfacc1c5755b76868dc4cfef39c&type=album)



На всякий случай: это VAE, то есть, latent у него все еще состоит из mu и sigma

Таким образом, при генерации новой рандомной картинки мы должны будем передать декодеру сконкатенированные латентный вектор и класс картинки.

P.S. Также можно передавать класс картинки не только в первый слой, но и в каждый слой сети. То есть на каждом слое конкатенировать выход из предыдущего слоя и информацию о классе.

In [128]:

class CVAE(nn.Module):
    def __init__(self):
        super(CVAE, self).__init__()
        '''<определите архитектуры encoder и decoder
        помните, у encoder должны быть два "хвоста", 
        т.е. encoder должен кодировать картинку в 2 переменные -- mu и logsigma>'''

        self.encoder = nn.Sequential(
                nn.Conv2d(1, 32, stride=(1, 1), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Conv2d(32, 64, stride=(2, 2), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Conv2d(64, 64, stride=(2, 2), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Conv2d(64, 64, stride=(1, 1), kernel_size=(3, 3), padding=1),
                nn.Flatten(),
        )
        self.z_mean = torch.nn.Linear(3136, 2)
        self.z_log_var = torch.nn.Linear(3136, 2)
        #mu = self.z_mean(self.encoder(x))
        #logsigma = self.z_log_var(self.encoder(x))
        
        '''<реализуйте forward проход энкодера
        в качестве ваозвращаемых переменных -- mu и logsigma>'''
        
        self.decoder = nn.Sequential(
                torch.nn.Linear(12, 3136),
                Reshape(-1, 64, 7, 7),
                nn.ConvTranspose2d(64, 64, stride=(1, 1), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.ConvTranspose2d(64, 64, stride=(2, 2), kernel_size=(3, 3), padding=1),                
                nn.LeakyReLU(0.01),
                nn.ConvTranspose2d(64, 32, stride=(2, 2), kernel_size=(3, 3), padding=0),                
                nn.LeakyReLU(0.01),
                nn.ConvTranspose2d(32, 1, stride=(1, 1), kernel_size=(3, 3), padding=0), 
                Trim(),  # 1x29x29 -> 1x28x28

                nn.Sigmoid()
                )
        #reconstruction = self.decoder(z)
    
    def gaussian_sampler(self, mu, logsigma):
        if self.training:
            eps = torch.randn(mu.size(0), mu.size(1)).to(device)
            z = mu + eps * torch.exp(logsigma/2.) 
            return z
            #<засемплируйте латентный вектор из нормального распределения с параметрами mu и sigma>
        else:
            # на инференсе возвращаем не случайный вектор из нормального распределения, а центральный -- mu. 
            # на инференсе выход автоэнкодера должен быть детерминирован.
            return mu
    
    def one_hot(self, labels):
        targets = torch.zeros(labels.size(0), 10)
        for i, label in enumerate(labels):
            targets[i, label] = 1
        return targets.to(device)
       #<реализуйте forward проход декодера
        #в качестве возвращаемой переменной -- reconstruction>

    def forward(self, x, labels):
        mu, logsigma = self.z_mean(self.encoder(x)), self.z_log_var(self.encoder(x))
        gaus_sample = self.gaussian_sampler(mu, logsigma)
        one_hot_vector = self.one_hot(labels)
        reconstruction = self.decoder(torch.cat([gaus_sample, one_hot_vector], 1))#decode(gaussian_sampler(mu, logsigma))
        #<используя encode и decode, реализуйте forward проход автоэнкодера
        #в качестве ваозвращаемых переменных -- mu, logsigma и reconstruction>
        return mu, logsigma, reconstruction

In [129]:
criterion = loss_vae
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

autoencoder = CVAE().to(device)
#print(autoencoder)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr = 0.00005)

In [130]:
n_epochs = 25
train_losses = []
val_losses = []
#train_losses_per_epoch = []
#val_losses_per_epoch = []
for epoch in tqdm(range(n_epochs)):
    autoencoder.train()
    train_losses_per_epoch = []
    for i, (batch, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        #print(type(batch))
        mu, logsigma, reconstruction = autoencoder(torch.tensor(batch).to(device), torch.tensor(labels).to(device))
        #print(reconstruction.shape, mu.shape, logsigma.shape, end = '\n' )
        #reconstruction = reconstruction.view(32,1,28,28)#(-1, 28, 28, 3)
        loss = criterion(batch.to(device).float(), mu, logsigma, reconstruction)
        loss.backward()
        optimizer.step()
        train_losses_per_epoch.append(loss.item())
    #train_losses.append(train_losses_per_epoch/ 32)
    train_losses.append(np.mean(train_losses_per_epoch))

    autoencoder.eval()
    val_losses_per_epoch = []
    with torch.no_grad():
        for i, (batch, label) in enumerate(val_loader):
            mu, logsigma, reconstruction = autoencoder(torch.tensor(batch).to(device), torch.tensor(label).to(device))
            #reconstruction = reconstruction.view(-1,1,28,28)
            loss = criterion(batch.to(device).float(), mu, logsigma, reconstruction)
            val_losses_per_epoch.append(loss.item())

    val_losses.append(np.mean(val_losses_per_epoch))

In [131]:
epochs__ = range(1,26)
plt.plot(epochs__, train_losses, 'g', label='Training loss')
plt.plot(epochs__, val_losses, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Sampling


Тут мы будем сэмплировать из CVAE. Это прикольнее, чем сэмплировать из простого AE/VAE: тут можно взять один и тот же латентный вектор и попросить CVAE восстановить из него картинки разных классов!
Для MNIST вы можете попросить CVAE восстановить из одного латентного вектора, например, картинки цифры 5 и 7.

In [132]:
cvae_latent = []
with torch.no_grad():
    for i, (batch, labels) in enumerate(train_loader):
        batch = batch.to(device)
        mu, logsigma = autoencoder.z_mean(autoencoder.encoder(batch)), autoencoder.z_log_var(autoencoder.encoder(batch))
        cvae_lat = autoencoder.gaussian_sampler(mu, logsigma)
        cvae_latent.extend(cvae_lat)
        
        
'''<тут нужно научиться сэмплировать из декодера цифры определенного класса>'''

In [133]:
cvae_random = np.random.choice(60000,15)
print(cvae_random)

In [134]:
one_5 = torch.tensor([0,0,0,0,0,1,0,0,0,0]).to(device)
one_7 = torch.tensor([0,0,0,0,0,0,0,1,0,0]).to(device)
save_5, save_7 = [], []
with torch.no_grad():
    for i in cvae_random:
        recon_5 = autoencoder.decoder(torch.cat([cvae_latent[i] , one_5]))
        recon_7 = autoencoder.decoder(torch.cat([cvae_latent[i] , one_7]))
        save_5.extend(recon_5.cpu().detach().numpy())
        save_7.extend(recon_7.cpu().detach().numpy())

In [135]:
save_5, save_7 = np.array(save_5), np.array(save_7)
plt.figure(figsize=(8, 20))
for i, (res, gt) in enumerate(zip(save_5[:15], save_7[:15])):
    plt.subplot(15, 2, 2*i+1)
    plt.imshow(gt[0])
    plt.subplot(15, 2, 2*i+2)
    plt.imshow(res[0])


Splendid! Вы великолепны!


### Latent Representations

Давайте посмотрим, как выглядит латентное пространство картинок в CVAE и сравним с картинкой для VAE =)

Опять же, нужно покрасить точки в разные цвета в зависимости от класса.

In [139]:
tsne_latent_cvae = []
with torch.no_grad():
    for i, (batch, labels) in enumerate(val_loader):
        batch, labels = batch.to(device), labels.to(device)
        mu, logsigma = autoencoder.z_mean(autoencoder.encoder(batch.to(device))), autoencoder.z_log_var(autoencoder.encoder(batch.to(device)))
        latent_vector = autoencoder.gaussian_sampler(mu, logsigma)
        #print(latent_vector.size(), autoencoder.one_hot(labels).size())
        #cat_cvae = torch.cat([latent_vector, autoencoder.one_hot(labels)], 1)
        tsne_latent_cvae.extend(latent_vector.cpu().detach().numpy())

In [140]:
from sklearn.manifold import TSNE
X_embedded_cvae = TSNE(n_components=2).fit_transform(tsne_latent_cvae)

In [141]:
d_cvae = [test_dataset[i][1] for i in range(len(test_dataset))]
import pandas as pd
import seaborn as sns
df = pd.DataFrame()
df["y"] = d_cvae
df["comp-1"] = X_embedded_cvae[:,0]
df["comp-2"] = X_embedded_cvae[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 10),
                data=df).set(title="MNIST data T-SNE projection")

Что вы думаете насчет этой картинки? Отличается от картинки для VAE?

# BONUS 1: Denoising

## Внимание! За бонусы доп. баллы не ставятся, но вы можете сделать их для себя.

У автоэнкодеров, кроме сжатия и генерации изображений, есть другие практические применения. Про одно из них эта бонусная часть задания.

Автоэнкодеры могут быть использованы для избавления от шума на фотографиях (denoising). Для этого их нужно обучить специальным образом: input картинка будет зашумленной, а выдавать автоэнкодер должен будет картинку без шума. 
То есть, loss-функция AE останется той же (MSE между реальной картинкой и выданной), а на вход автоэнкодеру будет подаваться зашумленная картинка.

<a href="https://ibb.co/YbRJ1nZ"><img src="https://i.ibb.co/0QD164t/Screen-Shot-2020-06-04-at-4-49-50-PM.png" alt="Screen-Shot-2020-06-04-at-4-49-50-PM" border="0"></a>

Для этого нужно взять ваш любимый датасет (датасет лиц из первой части этого задания или любой другой) и сделать копию этого датасета с шумом. 

В питоне шум можно добавить так:

In [ ]:
noise_factor = 0.5
X_noisy = X + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X.shape) 

In [ ]:
<тут ваш код обучения автоэнкодера на зашумленных картинках. Не забудтье разбить на train/test!>

In [ ]:
<тут проверка, как AE убирает щум с тестовых картинок. Надеюсь, все получилось =)>

# BONUS 2: Image Retrieval

## Внимание! За бонусы доп. баллы не ставятся, но вы можете сделать их для себя.

Давайте представим, что весь наш тренировочный датасет -- это большая база данных людей. И вот мы получили картинку лица какого-то человека с уличной камеры наблюдения (у нас это картинка из тестового датасета) и хотим понять, что это за человек. Что нам делать? Правильно -- берем наш VAE, кодируем картинку в латентное представление и ищем среди латентныз представлений лиц нашей базы самые ближайшие!

План:

1. Получаем латентные представления всех лиц тренировочного датасета
2. Обучаем на них LSHForest `(sklearn.neighbors.LSHForest)`, например, с `n_estimators=50`
3. Берем картинку из тестового датасета, с помощью VAE получаем ее латентный вектор
4. Ищем с помощью обученного LSHForest ближайшие из латентных представлений тренировочной базы
5. Находим лица тренировочного датасета, которым соответствуют ближайшие латентные представления, визуализируем!

Немного кода вам в помощь: (feel free to delete everything and write your own)

In [ ]:
codes = <поучите латентные представления картинок из трейна>

In [ ]:
# обучаем LSHForest
from sklearn.neighbors import LSHForest
lshf = LSHForest(n_estimators=50).fit(codes)

In [ ]:
def get_similar(image, n_neighbors=5):
  # функция, которая берет тестовый image и с помощью метода kneighbours у LSHForest ищет ближайшие векторы
  # прогоняет векторы через декодер и получает картинки ближайших людей

  code = <получение латентного представления image>
    
  (distances,),(idx,) = lshf.kneighbors(code, n_neighbors=n_neighbors)

  return distances, X_train[idx]

In [ ]:
def show_similar(image):

  # функция, которая принимает тестовый image, ищет ближайшие к нему и визуализирует результат
    
    distances,neighbors = get_similar(image,n_neighbors=11)
    
    plt.figure(figsize=[8,6])
    plt.subplot(3,4,1)
    plt.imshow(image.cpu().numpy().transpose([1,2,0]))
    plt.title("Original image")
    
    for i in range(11):
        plt.subplot(3,4,i+2)
        plt.imshow(neighbors[i].cpu().numpy().transpose([1,2,0]))
        plt.title("Dist=%.3f"%distances[i])
    plt.show()

In [ ]:
<тут выведите самые похожие лица к какому-нибудь лицу из тестовой части датасета>